In [1]:
import os
import pytesseract as tess
import tkinter as tk
from tkinter import filedialog, StringVar, IntVar
import pandas as pd
import cv2
from pytesseract import Output
import csv


tess.pytesseract.tesseract_cmd = 'C:/Program Files (x86)/Tessaract/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "C:/Program Files (x86)/Tessaract/tessdata"'
root = tk.Tk()

# reading image using opencv
image = cv2.imread('1.PNG')

#converting image into gray scale image
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# converting it to binary image by Thresholding
# this step is require if you have colored image because if you skip this part
# then tesseract won't able to detect text correctly and this will give incorrect result
threshold_img = cv2.threshold(gray_image, 155, 255, cv2.THRESH_TOZERO | cv2.THRESH_BINARY)[1]

# display image
cv2.imshow('gray image', threshold_img)

# Maintain output window until user presses a key, 0 indicates indefinite wait unless closed by the programmer. other values are measured in miliseconds
cv2.waitKey(0)

# Destroying present windows on screen
cv2.destroyAllWindows()


In [2]:
#configuring parameters for tesseract
# now feeding image to tesseract
details = tess.image_to_data(threshold_img, output_type=Output.DICT, config=tessdata_dir_config)
print(details)

{'level': [1, 2, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5], 'page_num': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'block_num': [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'par_num': [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [3]:
n_boxes = len(details['text'])
for i in range(n_boxes):
    if int(details['conf'][i]) > 20:
        (x, y, w, h) = (details['left'][i], details['top'][i], details['width'][i], details['height'][i])
        threshold_img = cv2.rectangle(threshold_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('captured text', threshold_img)
cv2.waitKey(0)

# Destroying present windows on screen
cv2.destroyAllWindows()

In [4]:
parse_text = []
word_list = []
last_word = []

for word in details['text']:
    if word!='':
        word_list.append(word)
        last_word = word
    if (last_word!='' and word == '') or (word==details['text'][-1]):
        parse_text.append(word_list)
        word_list = []
        

In [5]:
with open ('results.txt', 'w', newline='') as file:
    csv.writer(file, delimiter = " ").writerows(parse_text)